In [59]:
import polars as pl
from fastauc.fastauc.fast_auc import CppAuc
import numpy as np

In [60]:
cpp_auc = CppAuc()
def get_score(df):
    df = df.select(['impression_id','article','target','prediction']).group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
    
    result = np.mean(
            [cpp_auc.roc_auc_score(np.array(y_t).astype(bool), np.array(y_s).astype(np.float32)) 
                for y_t, y_s in zip(df['target'].to_list(), 
                                    df['prediction'].to_list())]
        )
    print(result)

In [61]:
predictions = pl.read_parquet('/home/ubuntu/experiments/analyze_prediction/scores_validation_stacking.parquet')
print(predictions)
get_score(predictions)


shape: (2_928_942, 390)
┌────────────┬─────────┬────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ impression ┆ article ┆ target ┆ prediction ┆ … ┆ roberta_em ┆ w_2_vec_em ┆ emotions_ ┆ constrast │
│ _id        ┆ ---     ┆ ---    ┆ ---        ┆   ┆ b_icm_minu ┆ b_icm_minu ┆ emb_icm_m ┆ ive_emb_i │
│ ---        ┆ i32     ┆ i8     ┆ f64        ┆   ┆ s_median_a ┆ s_median_a ┆ inus_medi ┆ cm_minus_ │
│ u32        ┆         ┆        ┆            ┆   ┆ …          ┆ …          ┆ an_…      ┆ med…      │
│            ┆         ┆        ┆            ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│            ┆         ┆        ┆            ┆   ┆ f32        ┆ f32        ┆ f32       ┆ f32       │
╞════════════╪═════════╪════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 96791      ┆ 9783865 ┆ 0      ┆ 0.075048   ┆ … ┆ -1.846875  ┆ -0.003178  ┆ -0.00133  ┆ -0.001478 │
│ 96791      ┆ 9784591 ┆ 0      ┆ 0.19992    ┆ … ┆ -0.00018   ┆ -0.

In [62]:
for col in predictions.columns:
    print(col)

impression_id
article
target
prediction
prediction_catboost_ranker
prediction_catboost_classifier
prediction_dcn
prediction_GANDALF
prediction_mlp
prediction_wd
normalized_prediction_catboost_ranker
normalized_prediction_catboost_classifier
normalized_prediction_dcn
normalized_prediction_GANDALF
normalized_prediction_mlp
normalized_prediction_wd
art_norm_prediction_catboost_ranker
art_norm_prediction_catboost_classifier
art_norm_prediction_dcn
art_norm_prediction_GANDALF
art_norm_prediction_mlp
art_norm_prediction_wd
prediction_hybrid
mean_prediction_catboost_ranker
mean_prediction_catboost_classifier
mean_prediction_dcn
mean_prediction_GANDALF
mean_prediction_mlp
mean_prediction_wd
mean_prediction_hybrid
skew_prediction_catboost_ranker
skew_prediction_catboost_classifier
skew_prediction_dcn
skew_prediction_GANDALF
skew_prediction_mlp
skew_prediction_wd
skew_prediction_hybrid
std_prediction_catboost_ranker
std_prediction_catboost_classifier
std_prediction_dcn
std_prediction_GANDALF
std

In [75]:
query = 'delay'

for col in predictions.columns:
    if query in col:
        print(col)

article_delay_days
article_delay_hours
mean_topics_mean_delay_days
mean_topics_mean_delay_hours
user_mean_delay_days
user_mean_delay_hours
article_delay_hours_l_inf_impression
article_delay_hours_minus_median_impression
std_impression_article_delay_hours
skew_impression_article_delay_hours
entropy_impression_article_delay_hours
article_delay_hours_rank_impression
mean_topics_mean_delay_hours_rank_impression
article_delay_hours_l_inf_article


In [68]:
get_score(predictions.with_columns(
        pl.when(pl.col('article_delay_days') > 3)\
            .then(pl.col('prediction') * 0.5)\
            .otherwise(pl.col('prediction'))
    ).with_columns(
        pl.when(pl.col('article_delay_hours') > pl.col('user_mean_delay_hours')*3)\
            .then(pl.col('prediction') * 1.2)\
            .otherwise(pl.col('prediction'))
    ).with_columns(
        pl.when(pl.col('normalized_endorsement_10h') == pl.col('normalized_endorsement_10h').max().over('impression_id'))
            .then(pl.col('prediction') * 0.95)\
            .otherwise(pl.col('prediction'))
    )
)

0.8250008049549922


In [65]:
behaviors_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/behaviors.parquet')
history_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/history.parquet')
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/articles.parquet')

In [67]:
hf = history_train.select(['user_id', 'article_id_fixed']).explode('article_id_fixed').rename({'article_id_fixed': 'article_id'})\
    .join(articles.select(['article_id','total_inviews', 'total_pageviews']).fill_null(0), on='article_id')\
    .filter(pl.col('total_inviews') > 0)\
    .with_columns(
        (pl.col('total_pageviews') / pl.col('total_inviews')).alias('total_pageviews/inviews')
    )\
    .group_by('user_id').agg(
        pl.col('total_pageviews/inviews').mean().alias('history_total_pageviews/inviews_avg'),
            pl.col('total_pageviews/inviews').std().alias('history_total_pageviews/inviews_std'),
            pl.col('total_pageviews/inviews').median().alias('history_total_pageviews/inviews_median'),
    )
predictions = predictions.join(hf, on='user_id', how = 'left')

In [69]:
predictions

impression_id,article,target,prediction,prediction_catboost_ranker,prediction_catboost_classifier,prediction_dcn,prediction_GANDALF,prediction_mlp,prediction_wd,normalized_prediction_catboost_ranker,normalized_prediction_catboost_classifier,normalized_prediction_dcn,normalized_prediction_GANDALF,normalized_prediction_mlp,normalized_prediction_wd,art_norm_prediction_catboost_ranker,art_norm_prediction_catboost_classifier,art_norm_prediction_dcn,art_norm_prediction_GANDALF,art_norm_prediction_mlp,art_norm_prediction_wd,prediction_hybrid,mean_prediction_catboost_ranker,mean_prediction_catboost_classifier,mean_prediction_dcn,mean_prediction_GANDALF,mean_prediction_mlp,mean_prediction_wd,mean_prediction_hybrid,skew_prediction_catboost_ranker,skew_prediction_catboost_classifier,skew_prediction_dcn,skew_prediction_GANDALF,skew_prediction_mlp,skew_prediction_wd,skew_prediction_hybrid,…,std_article_distilbert_emb_icm,std_article_bert_emb_icm,std_article_roberta_emb_icm,std_article_w_2_vec_emb_icm,std_article_emotions_emb_icm,std_article_constrastive_emb_icm,skew_article_kenneth_emb_icm,skew_article_distilbert_emb_icm,skew_article_bert_emb_icm,skew_article_roberta_emb_icm,skew_article_w_2_vec_emb_icm,skew_article_emotions_emb_icm,skew_article_constrastive_emb_icm,kurtosis_article_kenneth_emb_icm,kurtosis_article_distilbert_emb_icm,kurtosis_article_bert_emb_icm,kurtosis_article_roberta_emb_icm,kurtosis_article_w_2_vec_emb_icm,kurtosis_article_emotions_emb_icm,kurtosis_article_constrastive_emb_icm,entropy_article_kenneth_emb_icm,entropy_article_distilbert_emb_icm,entropy_article_bert_emb_icm,entropy_article_roberta_emb_icm,entropy_article_w_2_vec_emb_icm,entropy_article_emotions_emb_icm,entropy_article_constrastive_emb_icm,kenneth_emb_icm_minus_median_article,distilbert_emb_icm_minus_median_article,bert_emb_icm_minus_median_article,roberta_emb_icm_minus_median_article,w_2_vec_emb_icm_minus_median_article,emotions_emb_icm_minus_median_article,constrastive_emb_icm_minus_median_article,history_total_pageviews/inviews_avg,history_total_pageviews/inviews_std,history_total_pageviews/inviews_median
u32,i32,i8,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,…,f32,f32,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64
96791,9783865,0,0.075048,-0.666797,0.082062,0.043175,0.14195,0.060346,0.057008,0.0,0.0,0.0,0.0,0.0,0.0,0.324926,0.088514,0.045872,0.153701,0.067696,0.062322,0.0,0.220082,0.253237,0.350458,0.343151,0.367336,0.337175,0.543405,-0.262577,0.296461,-0.821649,0.419907,-0.58474,-1.18355,-0.353738,…,6.779265,0.030801,3.140027,0.006229,0.017181,0.002783,1.272064,1.046977,1.088959,1.297803,1.29522,1.078533,1.461609,1.638777,0.643103,0.767744,1.485673,1.405132,0.826377,2.443438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000422,-2.730756,-0.024237,-1.846875,-0.003178,-0.00133,-0.001478,0.205572,0.053941,0.204381
96791,9784591,0,0.19992,-0.201826,0.24496,0.496269,0.278617,0.492531,0.376784,0.27914,0.44268,0.957946,0.303862,0.834344,0.744446,0.400545,0.24594,0.499679,0.278779,0.498223,0.378448,0.435855,0.220082,0.253237,0.350458,0.343151,0.367336,0.337175,0.543405,-0.262577,0.296461,-0.821649,0.419907,-0.58474,-1.18355,-0.353738,…,7.509755,0.047205,6.185093,0.010945,0.016172,0.004107,1.588415,1.063038,1.128102,1.475852,2.023667,1.084642,2.014237,2.528819,0.664436,0.903211,2.306099,5.010594,0.659787,4.696737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002441,0.977748,-0.021208,-0.00018,-0.001672,-0.003989,-0.001263,0.205572,0.053941,0.204381
96791,9784679,0,0.306644,0.490838,0.283759,0.408114,0.406924,0.437615,0.381464,0.694972,0.548115,0.771566,0.589136,0.728329,0.75534,0.451658,0.285161,0.414432,0.407297,0.448214,0.385264,0.696486,0.220082,0.253237,0.350458,0.343151,0.367336,0.337175,0.543405,-0.262577,0.296461,-0.821649,0.419907,-0.58474,-1.18355,-0.353738,…,8.403144,0.040637,5.682543,0.010008,0.0

In [74]:
get_score(predictions.with_columns(
        pl.when(pl.col('article_delay_days') > 3)\
            .then(pl.col('prediction') * 0.5)\
            .otherwise(pl.col('prediction'))
    ).with_columns(
        pl.when(pl.col('article_delay_hours') > pl.col('user_mean_delay_hours')*3)\
            .then(pl.col('prediction') * 1.2)\
            .otherwise(pl.col('prediction'))
    ).with_columns(
        pl.when(pl.col('normalized_endorsement_10h') == pl.col('normalized_endorsement_10h').max().over('impression_id'))
            .then(pl.col('prediction') * 0.95)\
            .otherwise(pl.col('prediction'))
    ))

0.8249911278934282
